# 10-714 Homework 4

In this homework, you will leverage all of the components built in the last three homeworks to solve some modern problems with high performing network structures. We will start by adding a few new ops leveraging our new CPU/CUDA backends. Then, you will implement convolution, and a convolutional neural network to train a classifier on the CIFAR-10 image classification dataset. Finally, you will implement recurrent and long-short term memory (LSTM) neural networks, and do word-level prediction language modeling on the *Penn Treebank* dataset.

As always, we will start by copying this notebook and getting the starting code. Reminder: __you must save a copy in drive__.

In [ ]:
# Code to set up the assignment
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
!mkdir -p 10714
%cd /content/drive/MyDrive/10714
!git clone https://github.com/dlsys10714/hw4.git
%cd /content/drive/MyDrive/10714/hw4

In [ ]:
!pip3 install --upgrade --no-deps git+https://github.com/dlsys10714/mugrade.git
!pip3 install pybind11

Download the datasets you will be using for this assignment.

In [1]:
!python3 ./apps/download_dataset.py

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1


To finish setting up the assignment, go ahead and fill in all the code between `### BEGIN YOUR SOLUTION` and `### END YOUR SOLUTION` using your solution code from previous homework.

In [54]:
!make

-- Found pybind11: /home/willyu/anaconda3/envs/10-414/lib/python3.9/site-packages/pybind11/include (found version "2.10.0")
-- Found cuda, building cuda backend
Wed Jan 11 13:42:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:06:00.0 Off |                  N/A |
| 30%   37C    P8    17W / 350W |      1MiB / 24268MiB |      0%      Default |
|                               |                      |                  N/A |
+------

In [1]:
import sys
sys.path.append('./python')

## Part 1: ND Backend [10 pts]

Fill in the following classes in `python/needle/ops.py`:

- `PowerScalar`
- `EWiseDiv`
- `DivScalar`
- `Transpose`
- `Reshape`
- `BroadcastTo`<br>
  Call `compact()` before NDArray `broadcast_to()`, see [here](https://forum.dlsyscourse.org/t/test-ndarray-backend-with-test-optim-py-hw2/2821/3).
- `Summation`<br>
  `summation()` is called in the backward pass of `BroadcastTo`. Thus, its backend should support multi-axes reduction, which is not true in Homework 3. The call stack of `summation()` forward pass is
  |     |     |
  | --- | --- |
  | `ops.py` | `summation` |
  | `autograd.py` | `TensorOp.__call__` |
  | `autograd.py` | `Tensor.make_from_op` |
  | `autograd.py` |`Value.realize_cached_data` |
  | `ops.py` | `Summation.compute`|
  | `backend_ndarray/ndarray.py` | `summation` |
  | `backend_ndarray/ndarray.py` | `NDArray.sum` |
  | `backend_ndarray/ndarray_backend_*.*` | `reduce_sum` |
- `MatMul`
- `Negate`
- `Log`
- `Exp`
- `ReLU`
- `LogSumExp`
- `Tanh` (new)<br>
  $$
  \frac{d \tanh}{d x} = 1 - (\tanh x)^2
  $$
- `Stack` (new)<br>
  See [this post](https://forum.dlsyscourse.org/t/stack-op-could-someone-share-some-idea-about-how-to-implement-stack-op/2804) for implementation hints. Note that the `compute` method of an operator takes NDArray (`Value.cached_data`) as inputs. The above call stack is an example.<br><br>
  The operation is illustrated in the following simplified setting:
  ```python
      a = [[1,2],    b = [[5,6],    c = [[ 9,10],
           [3,4]]         [7,8]]         [11,12]]
      ---
      stack(args=(a,b,c),
            axis=1)
      ---
      # shape: (2, 3, 2)
      [[[ 1, 2],
        [ 5, 6],
        [ 9,10]],

       [[ 3, 4],
        [ 7, 8],
        [11,12]]]
  ```
  The indices of elements of `a`, `b`, and `c` are `[slice(None), 0, slice(None)]`, `slice(None), 1, slice(None)`, and `slice(None), 2, slice(None)`, respectively. That is, `Stack` simply <font color="blue">inserts a position</font>, specified by `axis`, <font color="blue">to the indices</font> of NDArray/Tensor elements.
- `Split` (new)

Note that for most of these, you already wrote the solutions in the previous homework, and you do not need to change your previous solution. However, `Tanh`, `Stack`, and `Split` are newly added. `Stack` concatenates same-sized tensors along a new axis, and `Split` undoes this operation. The gradients of the two operations can be written <font color="blue">in terms of each other</font>. We do not directly test `Split`, and only test the backward pass of `Stack` (for which we assume you used `Split`).

**Note:** You may want to make `Summation` support sums over multiple axes; you will likely need it for the backward pass of the BroadcastTo op if yours supports broadcasting over multiple axes at a time. However, this is more about ease of use than necessity, and we leave this decision up to you (there are no corresponding tests).

**Note:** Depending on your implementations, you may want to ensure that you **call `compact()` before reshaping arrays**. (If this is necessary, you will run into corresponding error messages later in the assignment.)

In [106]:
!python3 -m pytest -l -v -k "nd_backend"

============================= test session starts ==============================
platform linux -- Python 3.9.13, pytest-7.1.3, pluggy-1.0.0 -- /home/willyu/anaconda3/envs/10-414/bin/python3
cachedir: .pytest_cache
rootdir: /home/willyu/10-414/hw4
plugins: anyio-3.6.1
collected 1741 items / 1623 deselected / 118 selected                          

tests/test_nd_backend.py::test_ewise_fn[cpu-shape0-divide] PASSED        [  0%]
tests/test_nd_backend.py::test_ewise_fn[cpu-shape0-subtract] PASSED      [  1%]
tests/test_nd_backend.py::test_ewise_fn[cpu-shape1-divide] PASSED        [  2%]
tests/test_nd_backend.py::test_ewise_fn[cpu-shape1-subtract] PASSED      [  3%]
tests/test_nd_backend.py::test_ewise_fn[cuda-shape0-divide] PASSED       [  4%]
tests/test_nd_backend.py::test_ewise_fn[cuda-shape0-subtract] PASSED     [  5%]
tests/test_nd_backend.py::test_ewise_fn[cuda-shape1-divide] PASSED       [  5%]
tests/test_nd_backend.py::test_ewise_fn[cuda-shape1-subtract] PASSED     [  6%]
tests/test

In [ ]:
!python3 -m mugrade submit "YOUR KEY HERE" -k "new_nd_backend"

## Part 2: CIFAR-10 dataset [10 points]

Next, you will write support for the [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) image classification dataset, which consists of $60,000$ $32 \times 32$ color images in $10$ classes, with $6,000$ images per class. There are $50$k training images and $10$k test images. 

Start by implementing the `__init__` method in `data/CIFAR10Dataset`. You can read in the link above how to properly read the CIFAR-10 dataset files you downloaded at the beginning of the homework. Also fill in `__getitem__` and `__len__`. Note that the return shape of the data from `__getitem__` should be in order $(3, 32, 32)$ (NCHW).

In [13]:
!python3 -m pytest -l -v -k "cifar10"

============================= test session starts ==============================
platform linux -- Python 3.9.13, pytest-7.1.3, pluggy-1.0.0 -- /home/willyu/anaconda3/envs/10-414/bin/python3
cachedir: .pytest_cache
rootdir: /home/willyu/10-414/hw4
plugins: anyio-3.6.1
collected 1803 items / 1791 deselected / 12 selected                           

tests/test_cifar_ptb_data.py::test_cifar10_dataset[True] PASSED          [  8%]
tests/test_cifar_ptb_data.py::test_cifar10_dataset[False] PASSED         [ 16%]
tests/test_cifar_ptb_data.py::test_cifar10_loader[cpu-True-1] PASSED     [ 25%]
tests/test_cifar_ptb_data.py::test_cifar10_loader[cpu-True-15] PASSED    [ 33%]
tests/test_cifar_ptb_data.py::test_cifar10_loader[cpu-False-1] PASSED    [ 41%]
tests/test_cifar_ptb_data.py::test_cifar10_loader[cpu-False-15] PASSED   [ 50%]
tests/test_cifar_ptb_data.py::test_cifar10_loader[cuda-True-1] PASSED    [ 58%]
tests/test_cifar_ptb_data.py::test_cifar10_loader[cuda-True-15] PASSED   [ 66%]
tests/test

In [ ]:
!python3 -m mugrade submit "YOUR KEY HERE" -k "cifar10"

## Part 3: Convolutional neural network [40 points]

Here's an outline of what you will do in this task.

In `backend_ndarray/ndarray.py`, implement:
- `flip`
- `pad`

In `ops.py`, implement (forward and backward):
- `Flip`
- `Dilate`
- `UnDilate`
- `Conv`

In `nn.py`, implement:
- `Flatten`
- `Conv`

In `../apps/models.py`, fill in the `ResNet9` class.  

In `../apps/simple_training.py`, fill in:
- `epoch_general_cifar10`,
- `train_cifar10`
- `evaluate_cifar10`

We have provided a `BatchNorm2d` implementation for you as a wrapper around your previous `BatchNorm1d` implementation. 

### Padding NDArrays

Convolution as typically implemented in deep learning libraries cuts down the size of inputs, e.g., a $(1, 32, 32, 3)$ image convolved with a $3 \times 3$ filter would give a $(1, 30, 30, \verb|c|)$ output. A way around this is to pad the input NDArray before performing convolution, e.g., pad with zeros to get a $(1, 34, 34, 3)$ NDArray so that the result remains $(1, 32, 32, \verb|c|)$. Padding is also <font color="red">required</font> for the <font color="blue">backward pass</font> of convolution.

You should implement `pad` in `ndarray.py` to closely reflect the behavior of [`np.pad`](https://numpy.org/doc/stable/reference/generated/numpy.pad.html). That is, `pad` should take a tuple of $2$-tuples with length equal to the number of dimensions of the array, where each element in the $2$-tuple corresponds to "left padding" and "right padding", respectively.

For example, if `A` is a $(10, 32, 32, 8)$ NDArray (NHWC), then `A.pad((0, 0), (2, 2), (2, 2), (0, 0))` would be a $(10, 36, 36, 8)$ NDArray where the "spatial" dimension has been padded by two zeros on all sides.

In [108]:
!python3 -m pytest -l -v -k "pad_forward"

============================= test session starts ==============================
platform linux -- Python 3.9.13, pytest-7.1.3, pluggy-1.0.0 -- /home/willyu/anaconda3/envs/10-414/bin/python3
cachedir: .pytest_cache
rootdir: /home/willyu/10-414/hw4
plugins: anyio-3.6.1
collected 1741 items / 1739 deselected / 2 selected                            

tests/test_conv.py::test_pad_forward[params0-needle.backend_ndarray.ndarray_backend_cpu] PASSED [ 50%]
tests/test_conv.py::test_pad_forward[params1-needle.backend_ndarray.ndarray_backend_cpu] PASSED [100%]

====================== 2 passed, 1739 deselected in 1.16s ======================


-------------------------------------

### Flipping NDArrays & FlipOp

In [26]:
import numpy as np
import ctypes

Some utility code for a demonstration below which you can probably ignore. It might be instructive to check out the `offset` function.

In [27]:
# reads off the underlying data array in order (i.e., offset 0, offset 1, ..., offset n)
# i.e., ignoring strides
def raw_data(X):
    X = np.array(X) # copy, thus compact X
    return np.frombuffer(ctypes.string_at(X.ctypes.data, X.nbytes), dtype=X.dtype, count=X.size)

# Xold and Xnew should reference the same underlying data
def offset(Xold, Xnew):
    assert Xold.itemsize == Xnew.itemsize
    # compare addresses to the beginning of the arrays
    return (Xnew.ctypes.data - Xold.ctypes.data)//Xnew.itemsize

def strides(X):
    return ', '.join([str(x//X.itemsize) for x in X.strides])

def format_array(X, shape):
    assert len(shape) == 3, "I only made this formatting work for ndims = 3"
    def chunks(l, n):
        n = max(1, n)
        return (l[i:i+n] for i in range(0, len(l), n))
    a = [str(x) if x >= 10 else ' ' + str(x) for x in X]
    a = ['(' + ' '.join(y) + ')' for y in [x for x in chunks(a, shape[-1])]]
    a = ['|' + ' '.join(y) + '|' for y in [x for x in chunks(a, shape[-2])]]
    return '  '.join(a)

def inspect_array(X, *, is_a_copy_of):
    # compacts X, then reads it off in order
    print('Data: %s' % format_array(raw_data(X), X.shape))
    # compares address of X to copy_of, thus finding X's offset
    print('Offset: %s' % offset(is_a_copy_of, X))
    print('Strides: %s' % strides(X))

In order to implement the backwards pass of $2$D convolution, we will (probably) need a function which flips axes of NDArrays. We say "probably" because you could probably cleverly implement your convolution forward function to avoid this. However, we think it is easiest to think about this if you have the ability to "flip" the kernel along its vertical and horizontal dimensions.

We will try to build up your intuition for the "flip" operation below in order to help you figure out how to implement it in `ndarray.py`. To do that, we explore NumPy [`np.flip`](https://numpy.org/doc/stable/reference/generated/numpy.flip.html) function below. One thing to note is that `flip` is typically implemented by using <font color="red">negative</font> strides and changing the <font color="blue">offset</font> of the underlying array.

For example, flipping an array on <font color="blue">all</font> of its axes is equivalent to reversing the array. In this case, you can imagine that we would want all the strides to be negative, and the offset to be the length of the array (to start at the end of the array and "stride" backwards).

Since we did <font color="red">not</font> explicitly support negative strides in our implementation for the last homework, we will merely call `NDArray.make` with them to make our "flipped" array and then immediately call `.compact()`. Other than changing unsigned ints to signed ints in a few places, we suspect your existing `compact` function should not have to change at all to accomodate negative strides. In the .cc and .cu files we distributed, we have already changed the function signatures to reflect this.

Alternatively, you could simply implement `flip` in the CPU backend by copying memory, which you <font color="blue">may</font> find more intuitive. We suggest following our mini tutorial below to keep your implementation Python-focused, since we believe it involves approximately the same amount of effort to implement it slightly more naively in C.

Use this array as reference for example:

In [28]:
A = np.arange(1, 25).reshape(3, 2, 4)
inspect_array(A, is_a_copy_of=A)

Data: |( 1  2  3  4) ( 5  6  7  8)|  |( 9 10 11 12) (13 14 15 16)|  |(17 18 19 20) (21 22 23 24)|
Offset: 0
Strides: 8, 4, 1


We have put brackets around each axis of the array. Notice that for this array, the offset is 0 and the strides are all positive.

---

See what happens when you flip the array along the last axis below. Note that the `inspect_array` function compacts the array after flipping it so you can see the "logical" order of the data, and the offset is calculated by comparing the address of the <font color="red">non</font>-compact flipped array with that of `is_copy_of`, i.e., the array `A` we looked at above.

That is, we are looking at how NumPy calculates the strides and offset for flipped arrays in order to copy this behavior in our own implementation.

In [29]:
inspect_array(np.flip(A, (2,)), is_a_copy_of=A)

Data: |( 4  3  2  1) ( 8  7  6  5)|  |(12 11 10  9) (16 15 14 13)|  |(20 19 18 17) (24 23 22 21)|
Offset: 3
Strides: 8, 4, -1


So flipping the last axis reverses the order of the elements within each $4$-dimensional "cell", as you can see above. The stride corresponding to the axis we flipped has been negated. And the offset is $3$ — this makes sense, e.g., because we want the new "first" element of the array to be $4$, which was at index $\verb|3|$ in `A`.

In [30]:
inspect_array(np.flip(A, (1,)), is_a_copy_of=A)

Data: |( 5  6  7  8) ( 1  2  3  4)|  |(13 14 15 16) ( 9 10 11 12)|  |(21 22 23 24) (17 18 19 20)|
Offset: 4
Strides: 8, -4, 1


Again for the middle axis: we negate the middle stride, and the offset is $4$, which seems reasonable since we now want the first element to be $5$, which was at index $\verb|4|$ in the original array `A`. 

In [31]:
inspect_array(np.flip(A, (0,)), is_a_copy_of=A)

Data: |(17 18 19 20) (21 22 23 24)|  |( 9 10 11 12) (13 14 15 16)|  |( 1  2  3  4) ( 5  6  7  8)|
Offset: 16
Strides: -8, 4, 1


Try to infer the general algorithm for computing the offset given the axis to flip.

`A` has strides $(\color{grey}{24, }8, 4, 1)$, where $24$ is the size of the entire array. If a <font color="blue">single</font> axis $i$ is flipped, the corresponding offset will be

$$
\begin{align*}
\verb|strides| &\longleftarrow (24,) \ \verb|concat| \ \verb|A.strides| \\
\verb|offset| &\longleftarrow \verb|strides|[i] - \verb|strides|[i+1]
\end{align*}
$$

---

Observe what happens when we flip <font color="blue">all</font> axes.

In [32]:
inspect_array(np.flip(A, (0,1,2)), is_a_copy_of=A)

Data: |(24 23 22 21) (20 19 18 17)|  |(16 15 14 13) (12 11 10  9)|  |( 8  7  6  5) ( 4  3  2  1)|
Offset: 23
Strides: -8, -4, -1


As mentioned earlier, the offset is then sufficient to point to the last element of the array, and this is just the "reverse order" version of `A`.

When we flip just $2$ axes…

In [33]:
inspect_array(np.flip(A, (0,1)), is_a_copy_of=A)

Data: |(21 22 23 24) (17 18 19 20)|  |(13 14 15 16) ( 9 10 11 12)|  |( 5  6  7  8) ( 1  2  3  4)|
Offset: 20
Strides: -8, -4, 1


In [34]:
inspect_array(np.flip(A, (0,2)), is_a_copy_of=A)

Data: |(20 19 18 17) (24 23 22 21)|  |(12 11 10  9) (16 15 14 13)|  |( 4  3  2  1) ( 8  7  6  5)|
Offset: 19
Strides: -8, 4, -1


Looking back on our previous offset computation, do you notice something?

Offset of multi-axes flipping amounts to the <font color="blue">sum</font> of correspnding offset of single-axis flipping.

---

With this exploration of NumPy `ndarray` flipping functionality, which uses negative strides and a custom offset, try to implement `flip` in `ndarray.py`. You also must implement `Flip` forward and backward functions in `ops.py`; note that these should be extremely short.

**Important:** You should call `NDArray.make` or `self.as_strided` with the new strides and offset, and then immediately `.compact()` this array. The result is then copied and has positive strides. We want this (less-than-optimal) behavior because we did <font color="red">not</font> account for negative strides in our previous implementation. It is OK to add a "flip" operator on the CPU/CUDA backend instead.

Aside: If you'd like to, consider where/if negative strides break your implementation; `__getitem__` definitely <font color="red">doesn't</font> work given how we process slices. is there anything else?

#### An implementation caveat

Flipping involves compacting an NDArray with <font color="red">negative</font> strides. If one simply copies her/his solution to the CPU/CUDA backend <font color="grey"> from homework 3</font> to this one, `TypeError` will occur. This is because the previous backend declares strides as `uint32_t`, which is <font color="red">in</font>compatible with negative values. Part of a solution is to modify NDArray backend, see [here](https://forum.dlsyscourse.org/t/q3-flip-typeerror-to-numpy-incompatible-function-arguments/2862/9).

In [107]:
!python3 -m pytest -l -v -k "flip"

============================= test session starts ==============================
platform linux -- Python 3.9.13, pytest-7.1.3, pluggy-1.0.0 -- /home/willyu/anaconda3/envs/10-414/bin/python3
cachedir: .pytest_cache
rootdir: /home/willyu/10-414/hw4
plugins: anyio-3.6.1
collected 1741 items / 1701 deselected / 40 selected                           

tests/test_conv.py::test_flip_forward[params0-needle.backend_ndarray.ndarray_backend_cpu] PASSED [  2%]
tests/test_conv.py::test_flip_forward[params0-needle.backend_ndarray.ndarray_backend_cuda] PASSED [  5%]
tests/test_conv.py::test_flip_forward[params1-needle.backend_ndarray.ndarray_backend_cpu] PASSED [  7%]
tests/test_conv.py::test_flip_forward[params1-needle.backend_ndarray.ndarray_backend_cuda] PASSED [ 10%]
tests/test_conv.py::test_flip_forward[params2-needle.backend_ndarray.ndarray_backend_cpu] PASSED [ 12%]
tests/test_conv.py::test_flip_forward[params2-needle.backend_ndarray.ndarray_backend_cuda] PASSED [ 15%]
tests/test_conv.py::tes

---

### Dilation

The dilation operator puts zeros between elements of an NDArray. We will need it for computing the <font color="blue">backward pass</font> of convolution when the stride of the convolution is greater than $1$. As an example, dilation should do the following to a $2 \times 2$ matrix when dilated by $1$ on both axes:

$$
\begin{bmatrix}
1 & 2 \\
3 & 4
\end{bmatrix}
\longrightarrow
\begin{bmatrix}
1 & 0 & 2 & 0 \\
0 & 0 & 0 & 0 \\
3 & 0 & 4 & 0 \\
0 & 0 & 0 & 0
\end{bmatrix}
$$

To get some intuition for why we need dilation for the backward pass of strided convolution, consider a convolution with `stride=2`, `padding="same"`, and `input_channels=output_channels=8` applied to an input of size $(10, 32, 32, 8)$. The resulting output will be of size $(10, 16, 16, 8)$ due to the stride, and thus `out_grad` will have shape $(10, 16, 16, 8)$. Yet, the gradient of the input needs to, of course, have shape $(10, 32, 32, 8)$ — so we have to increase the size of `out_grad` in some way. Consider also that you could implement <font color="blue">strided convolution</font> as `Conv(x)[:, ::2, ::2, :]`, i.e., only keeping every other pixel in the spatial dimension.


Implement `Dilate` in `ops.py`. This function takes two additional parameters (in attrs): the `dilation` amount and the `axes` to dilate. You must also implement the corresponding operator `UnDilate`, whose forward pass will be used to implement the gradient of `Dilate`. (This is why we do not have to implement `GetItem` and `SetItem`, which can be highly <font color="red">inefficient</font> to backpropagate without additional optimizations.)

In [109]:
!python3 -m pytest -l -v -k "dilate"

============================= test session starts ==============================
platform linux -- Python 3.9.13, pytest-7.1.3, pluggy-1.0.0 -- /home/willyu/anaconda3/envs/10-414/bin/python3
cachedir: .pytest_cache
rootdir: /home/willyu/10-414/hw4
plugins: anyio-3.6.1
collected 1741 items / 1715 deselected / 26 selected                           

tests/test_conv.py::test_dilate_forward[needle.backend_ndarray.ndarray_backend_cpu] PASSED [  3%]
tests/test_conv.py::test_dilate_forward[needle.backend_ndarray.ndarray_backend_cuda] PASSED [  7%]
tests/test_conv.py::test_dilate_backward[params0-needle.backend_ndarray.ndarray_backend_cpu] PASSED [ 11%]
tests/test_conv.py::test_dilate_backward[params0-needle.backend_ndarray.ndarray_backend_cuda] PASSED [ 15%]
tests/test_conv.py::test_dilate_backward[params1-needle.backend_ndarray.ndarray_backend_cpu] PASSED [ 19%]
tests/test_conv.py::test_dilate_backward[params1-needle.backend_ndarray.ndarray_backend_cuda] PASSED [ 23%]
tests/test_conv.py::tes

In [ ]:
!python3 -m mugrade submit "YOUR KEY HERE" -k "new_ops"

---

### Convolution: forward pass

Implement the forward pass of $2$D multi-channel convolution in `ops.py`. You should probably refer to [this notebook](https://github.com/dlsyscourse/public_notebooks/blob/main/convolution_implementation.ipynb) from lecture, which implements $2$D multi-channel convolution using im2col in NumPy.

**Note**: Your `Conv` op should accept tensors in the <font color="blue">NHWC</font> format, as in the example above, and weights in the format `(kernel_size, kernel_size, input_channels, output_channels)`.

However, you will need to add three additional features. Your convolution function should accept arguments for `stride` (default $1$), `padding` (default $0$), and `dilation` (default $1$). For `padding`, you should simply apply your padding function to the spatial dimensions (i.e., axes 1 and 2). Strided/dilated convolution should consist of a relatively small set of changes to your plain convolution implementation. We recommend implementing convolution <font color="red">without</font> stride or dilation first, ensuring you pass some of the tests below, and then adding in stride.

In [31]:
!python3 -m pytest -l -v -k "op_conv and forward"

============================= test session starts ==============================
platform linux -- Python 3.9.13, pytest-7.1.3, pluggy-1.0.0 -- /home/willyu/anaconda3/envs/10-414/bin/python3
cachedir: .pytest_cache
rootdir: /home/willyu/10-414/hw4
plugins: anyio-3.6.1
collected 353 items / 351 deselected / 2 selected                              

tests/test_conv.py::test_op_conv[forward-needle.backend_ndarray.ndarray_backend_cpu-Z_shape0-W_shape0-2-0-1] PASSED [ 50%]
tests/test_conv.py::test_op_conv[forward-needle.backend_ndarray.ndarray_backend_cuda-Z_shape0-W_shape0-2-0-1] PASSED [100%]

====================== 2 passed, 351 deselected in 1.59s =======================


---

### Gradient of convolution

Finding the gradients of $2$D multi-channel convolution can be technically quite challenging (especially "rigorously"). We will try to provide some useful hints here. Basically, we encourage you to make use of the surprising fact that <font color="blue">whatever makes the dimensions work out is typically right</font>.

Ultimately, the backward pass of convolution can be done in terms of the convolution operator itself, with some clever manipulations of `flip`, `dilate`, and multiple applications of `transpose` to both the arguments and the results. In the last section, we essentially implemented convolution as a matrix product: ignoring the various restride and reshape operations, we basically have something like `X @ W`, where `X` is the input and `W` is the weight. We also have `out_grad`, which is the same shape as `X @ W`. Now, you have already implemented the backward pass of matrix multiplication in a previous assignment, and we can use this knowledge to get some insight into the backward pass of convolution. In particular, referencing your `matmul` backward implementation, you may notice (heuristically speaking here):

```python
X.grad = out_grad @ W.transpose
W.grad = X.transpose @ out_grad
```

Surprisingly enough, things work out if we just assume that these are also convolutions (and now assuming that `out_grad`, `W`, and `X` are tensors <font color="green">amenable to</font> $2$D multi-channel convolution instead of matrices):

```
X.grad = ≈conv(≈out_grad, ≈W)
W.grad = ≈conv(≈X, ≈out_grad)
```

in which the `≈` indicates you need to apply some additional operators to these terms in order to get the dimensions to work out, such as permuting/transposing axes, dilating, changing the `padding=` argument, or permuting/transposing axes of the resulting convolution.

As we saw on the [last few slides here](https://dlsyscourse.org/slides/conv_nets.pdf) in class, the transpose of a convolution can be found by simply flipping the kernel. Since we're working in $2$D instead of $1$D, this means flipping the kernel both vertically and horizontally (thus why we implemented `flip`).

Summarizing some hints for both `X.grad` and `W.grad`:

`X.grad`
- The convolution of `out_grad` and `W`
  - `W` should be flipped over both the kernel dimensions
- If the convolution is strided, increase the size of `out_grad` with a corresponding dilation
  - Pay attention to the cases where $H$ or $W$ is <font color="red">not</font> neatly divisible by the stride.
- Do an example to analyze dimensions: note the shape you want for `X.grad`, and think about how you must permute/transpose the arguments and add padding to the convolution to achieve this shape
  - `out_grad` should be padded to simulate <font color="blue">*full correlation*</font>.
  - This padding depends on both the kernel size and the `padding` argument to the convolution

`W.grad`
- The convolution of `X` and `out_grad`
  - The gradients of `W` must be accumulated <font color="blue">over the batch dimension</font>.
  - Consider "turning batches into channels" via transpose/permute to let `conv` <font color="blue">itself</font> do such accumulation; see tips.
- Analyze dimensions: how can you modify `X` and `out_grad` so that the shape of their convolution matches the shape of `W`? You may need to transpose/permute the result.
  - Remember to account for the `padding` argument passed to convolution

General tips

- Deal with strided convolutions last (you should be able to just drop in `dilate` when you've passed most of the tests)
- Start with the case where `padding=0`, then consider changing `padding` arguments
- You can permute axes with multiple calls to `transpose`. Alternatively, define a `Permute` operator.<br>
  |     | `X` | `W` | `out_grad` |
  | --- | --- | --- | ---        |
  | Current shape | $$(N, H_\text{in}, W_\text{in}, C_\text{in})$$ | $$(K, K, C_\text{in}, C_\text{out})$$ | $$(N, H_\text{out}, W_\text{out}, C_\text{out})$$ |
  | Target shape | $$(C_\text{in}, H_\text{in}, W_\text{in}, N)$$ | $$(K^*, K^*, C_\text{out}, C_\text{in})$$ | $$(H_\text{out}, W_\text{out}, N, C_\text{out})$$ |
  | Useful in | $$\frac{\partial f}{\partial \mathbf{W}}$$ | $$\frac{\partial f}{\partial \mathbf{X}}$$ | $$\frac{\partial f}{\partial \mathbf{W}}$$ |
  | Operation | axes permutation | `transpose(W)` | axes permutation |
  
  A $*$ means the dimension may be padded  so that the value is <font color="red">inaccurate</font>.

It might also be useful to skip ahead to `nn.Conv`, pass the forward tests, and then use both the tests below and the `nn.Conv` backward tests to debug your implementation.

In [194]:
!python3 -m pytest -l -v -k "op_conv and backward" -s

============================= test session starts ==============================
platform linux -- Python 3.9.13, pytest-7.1.3, pluggy-1.0.0 -- /home/willyu/anaconda3/envs/10-414/bin/python3
cachedir: .pytest_cache
rootdir: /home/willyu/10-414/hw4
plugins: anyio-3.6.1
collected 1741 items / 1739 deselected / 2 selected                            

tests/test_conv.py::test_op_conv[backward-needle.backend_ndarray.ndarray_backend_cpu-Z_shape0-W_shape0-2-0-1] FAILED
tests/test_conv.py::test_op_conv[backward-needle.backend_ndarray.ndarray_backend_cuda-Z_shape0-W_shape0-2-0-1] FAILED

=================================== FAILURES ===================================
_ test_op_conv[backward-needle.backend_ndarray.ndarray_backend_cpu-Z_shape0-W_shape0-2-0-1] _

Z_shape = (1, 5, 5, 1), W_shape = (3, 3, 1, 1), stride = 2, padding = 0
dilation = 1, backward = True, device = cpu()

    @pytest.mark.parametrize("Z_shape, W_shape, stride, padding, dilation", op_conv_shapes)
    @pytest.mark.parametriz

---

### `nn.Conv`

#### Fixing `init._calculate_fans` for convolution

Previously, we have implemented Kaiming uniform/normal initializations, where we essentially assigned `fan_in = input_size` and `fan_out = output_size`.
For convolution, this becomes somewhat more detailed, in that you should multiply both of these by the "receptive field size", which is in this case just the product of the kernel sizes -- which in our case are always going to be the same, i.e., $k\times k$ kernels.

**You will need to edit your `kaiming_uniform`, etc. init functions to support multidimensional arrays.** In particular, you should add a new `shape` argument which is then passed to, e.g., the underlying `rand` function.

You can test this below; though it is not _directly_ graded, it must match ours to pass the nn.Conv mugrade tests.

In [ ]:
!python3 -m pytest -l -v -k "kaiming_uniform"

#### Implementing `nn.Conv`

Essentially, nn.Conv is just a wrapper of the convolution operator we previously implemented
which adds a bias term, initializes the weight and bias, and ensures that the padding is set so that the input and output dimensions are the same (in the `stride=1` case, anyways). 

Importantly, nn.Conv should support NCHW format instead of NHWC format. In particular, we think this makes more sense given our current BatchNorm implementation. You can implement this by applying `transpose` twice to both the input and output.  

- Ensure nn.Conv works for (N, C, H, W) tensors even though we implemented the conv op for (N, H, W, C) tensors
- Initialize the (k, k, i, o) weight tensor using Kaiming uniform initialization with default settings
- Initialize the (o,) bias tensor using uniform initialization on the interval $\pm$`1.0/(in_channels * kernel_size**2)**0.5`
- Calculate the appropriate padding to ensure input and output dimensions are the same
- Calculate the convolution, then add the properly-broadcasted bias term if present

You can now test your nn.Conv against PyTorch's nn.Conv2d with the two PyTest calls below.

In [ ]:
!python3 -m pytest -l -v -k "nn_conv_forward"

In [ ]:
!python3 -m mugrade submit "YOUR KEY HERE" -k "conv_forward"

In [ ]:
!python3 -m pytest -l -v -k "nn_conv_backward"

In [ ]:
!python3 -m mugrade submit "YOUR KEY HERE" -k "conv_backward"

---

#### ResNet9

You will now use your convolutional layer to implement a model similar to _ResNet9_, which is known to be a reasonable model for getting good accuracy on CIFAR-10 quickly (see [here](https://github.com/davidcpage/cifar10-fast)). Our main change is that we used striding instead of pooling and divided all of the channels by 4 for the sake of performance (as our framework is not as well-optimized as industry-grade frameworks).

In the figure below, before the linear layer, you should "flatten" the tensor. We have added a module called `Flatten` in `nn.py` that you can complete and use, or you can simply use `.reshape` in the `forward()` method of your ResNet9.

Make sure that you pass the device to all modules in your model; otherwise, you will get errors about mismatched devices when trying to run with CUDA.

<center><img src="./ResNet9.png" alt="ResNet9" width=500/></center>

We have tried to make it easier to pass the tests here than for previous assignments where you have implemented models. In particular, we are just going to make sure it has the right number of parameters and similar accuracy and loss after 1 or 2 batches of CIFAR-10.

In [ ]:
!python3 -m pytest -l -v -k "resnet9"

In [ ]:
!python3 -m mugrade submit "YOUR KEY HERE" -k "resnet9"

---

Now, you can train your model on CIFAR-10 using the following code. Note that this is likely going to be quite slow, and also  not all that accurate due to the lack of data augmentation. You should expect it to take around 500s per epoch.

In [ ]:
import sys
sys.path.append('./python')
sys.path.append('./apps')
import needle as ndl
from models import ResNet9
from simple_training import train_cifar10, evaluate_cifar10

device = ndl.cpu()
dataset = ndl.data.CIFAR10Dataset("data/cifar-10-batches-py", train=True)
dataloader = ndl.data.DataLoader(\
         dataset=dataset,
         batch_size=128,
         shuffle=True,
         collate_fn=ndl.data.collate_ndarray,
         device=device,
         dtype="float32")
model = ResNet9(device=device, dtype="float32")
train_cifar10(model, dataloader, n_epochs=10, optimizer=ndl.optim.Adam,
      lr=0.001, weight_decay=0.001)
evaluate_cifar10(model, dataloader)

## Part 4: Recurrent neural network [10 points]

In `nn.py`, implement `RNNCell`.

- $\boldsymbol{h}^\prime = \text{tanh}(\boldsymbol{x} \mathbf{W}_{ih} + \boldsymbol{b}_{ih} + \boldsymbol{h} \mathbf{W}_{hh} + \boldsymbol{b}_{hh})$. If nonlinearity is `'relu'`, then ReLU is used in place of tanh.
- All weights and biases should be initialized from $\mathcal{U}(-\sqrt{k}, \sqrt{k})$ where $k=\frac{1}{\text{hidden_size}}$.

In `nn.py`, implement `RNN`. For each element in the input sequence, each layer computes the following function:

$$
\boldsymbol{h}_t = \text{tanh}(\boldsymbol{x}_t \mathbf{W}_{ih} + \boldsymbol{b}_{ih} + \boldsymbol{h}_{t-1} \mathbf{W}_{hh} + \boldsymbol{b}_{hh})
$$

where $\boldsymbol{h}_t$ is the hidden state at time $t$, $\boldsymbol{x}_t$ is the input at time $t$, and $\boldsymbol{h}_{t-1}$ is the hidden state of the previous layer at time $t-1$ or the initial hidden state at time $0$. If nonlinearity is `'relu'`, then ReLU is used in place of tanh.

In a multi-layer RNN, the input $\boldsymbol{x}_t^{(l)}$ of the $l$-th layer ($l \ge 2$) is the hidden state $\boldsymbol{h}_t^{(l-1)}$ of the previous layer. When there are multiple layers, we first compute the hidden states of the first layer $\mathbf{H}^1$ over entire sequences, and then $\mathbf{H}^l$ serves as inputs to "deeper" layers.

### Implementation caveats

One may be tempted to index tensors, i.e., to use `__getitem__` or `__setitem__`. However, we have <font color="red">not</font> implemented these for tensors in Needle; instead, use `stack` and `split`. Likewise, simply indexing their output, a `TensorTuple`, is <font color="red">not</font> differentiable. Use `tuple_get_item` rather than list indices. This is because backpropagation (`….backward()`) applies to <font color="blue">all</font> leaf nodes in the computational graph, even though some of them should <font color="red">not</font> be gradient descended (`<optimizer>.step()`). For instance, sentences input `X` to language models are first split into a `TensorTuple` by sequence length, from which one token is taken at each time step. Do not use `…[t]` since the indexing operation is <font color="red">not</font> differentiable. We do <font color="red">not</font> optimize any input, but `X`, as a leaf node, does have a gradient when calling `….backward()`. This requires a <font color="blue">differentiable</font> indexing operation, `tuple_get_item(…, t)`.

This caveat applie to <font color="blue">all</font> the following sections.

Biases in PyTorch [`RNNCell`](https://pytorch.org/docs/stable/generated/torch.nn.RNNCell.html) has shape $($ `hidden_size` $,)$. I deliberately initialized `bias_ih` and `bias_hh` with shape $(1,$ `hidden_size` $)$ for the sake of less `reshape` operations. Therefore, I modified `test_rnn_cell` and `test_rnn` in [`test_sequence_models.py`](./tests/test_sequence_models.py) so that the reference `bias_ih` and `bias_hh` have shape $(1,$ `hidden_size` $)$.

In [110]:
!python3 -m pytest -l -v -k "test_rnn"

============================= test session starts ==============================
platform linux -- Python 3.9.13, pytest-7.1.3, pluggy-1.0.0 -- /home/willyu/anaconda3/envs/10-414/bin/python3
cachedir: .pytest_cache
rootdir: /home/willyu/10-414/hw4
plugins: anyio-3.6.1
collected 1741 items / 1101 deselected / 640 selected                          

tests/test_sequence_models.py::test_rnn_cell[cpu-tanh-True-True-1-1-1] PASSED [  0%]
tests/test_sequence_models.py::test_rnn_cell[cpu-tanh-True-True-1-1-15] PASSED [  0%]
tests/test_sequence_models.py::test_rnn_cell[cpu-tanh-True-True-1-11-1] PASSED [  0%]
tests/test_sequence_models.py::test_rnn_cell[cpu-tanh-True-True-1-11-15] PASSED [  0%]
tests/test_sequence_models.py::test_rnn_cell[cpu-tanh-True-True-12-1-1] PASSED [  0%]
tests/test_sequence_models.py::test_rnn_cell[cpu-tanh-True-True-12-1-15] PASSED [  0%]
tests/test_sequence_models.py::test_rnn_cell[cpu-tanh-True-True-12-11-1] PASSED [  1%]
tests/test_sequence_models.py::test_rnn_cell[c

In [ ]:
!python3 -m mugrade submit "YOUR KEY HERE" -k "rnn"

## Part 5: Long short-term memory network [10 points]
Implement - `Sigmoid`

$\sigma(x) = \frac{1}{1 + \text{exp}(-x)}$

In `python/needle/nn.py`, implement `Sigmoid`, `LSTMCell` and `LSTM`.

\begin{align}
i &= \sigma(xW_{ii} + b_{ii} + hW_{hi} + b_{hi}) \\
f &= \sigma(xW_{if} + b_{if} + hW_{hf} + b_{hf}) \\
g &= \text{tanh}(xW_{ig} + b_{ig} + hW_{hg} + b_{hg}) \\
o &= \sigma(xW_{io} + b_{io} + hW_{ho} + b_{ho}) \\
c^\prime &= f * c + i * g \\
h^\prime &= o * \text{tanh}(c^\prime)
\end{align}

where $\sigma$ is the sigmoid function, and $i$, $f$, $g$, $o$ are the input, forget, cell, and output gates, respectively. 

All weights and biases should be initialized from $\mathcal{U}(-\sqrt{k}, \sqrt{k})$ where $k=\frac{1}{\text{hidden_size}}$.

Now implement `LSTM` in `python/needle/nn.py`, which applies a multi-layer LSTM RNN to an input sequence. For each element in the input sequence, each layer computes the following function:

\begin{align}
i_t &= \sigma(x_tW_{ii} + b_{ii} + h_{(t-1)}W_{hi} + b_{hi}) \\
f_t &= \sigma(x_tW_{if} + b_{if} + h_{(t-1)}W_{hf} + b_{hf}) \\
g_t &= \text{tanh}(x_tW_{ig} + b_{ig} + h_{(t-1)}W_{hg} + b_{hg}) \\
o_t &= \sigma(x_tW_{io} + b_{io} + h_{(t-1)}W_{ho} + b_{ho}) \\
c_t &= f * c_{(t-1)} + i * g \\
h_t &= o * \text{tanh}(c_t)
\end{align},
where $h_t$ is the hidden state at time $t$, $c_t$ is the cell state at time $t$, $x_t$ is the input at time $t$, $h_{(t-1)}$ is the hidden state of the layer at time $t-1$ or the initial hidden state at time $0$, and $i_t$, $f_t$, $g_t$, $o_t$ are the input, forget, cell, and output gates at time $t$ respectively. 

In a multi-layer LSTM, the input $x_t^{(l)}$ of the $l$-th layer ($l \ge 2$) is the hidden state $h_t^{(l-1)}$ of the previous layer. 

In [ ]:
!python3 -m pytest -l -v -k "test_lstm"

In [ ]:
!python3 -m mugrade submit "YOUR KEY HERE" -k "lstm"

## Part 6: Penn Treebank dataset [10 points]

In word-level language modeling tasks, the model predicts the probability of the next word in the sequence, based on the words already observed in the sequence. You will write support for the Penn Treebank dataset, which consists of stories from the Wall Street Journal, to train and evaluate a language model on word-level prediction.

In `python/needle/data.py`, start by implementing the `Dictionary` class, which creates a dictionary from a list of words, mapping each word to a unique integer.

Next, we will use this `Dictionary` class to create a corpus from the train and test txt files in the Penn Treebank dataset that you downloaded at the beginning of the notebook. Implement the `tokenize` function in the `Corpus` class to do this.

In order to prepare the data for training and evaluation, you will next implement the `batchify` function. Starting from sequential data, batchify arranges the dataset into columns. For instance, with the alphabet as the sequence and batch size 4, we'd get

```
┌ a g m s ┐
│ b h n t │
│ c i o u │
│ d j p v │
│ e k q w │
└ f l r x ┘
```

These columns are treated as independent by the model, which means that the dependence of e. g. 'g' on 'f' cannot be learned, but allows more efficient batch processing.

Next, implement the `get_batch` function. `get_batch` subdivides the source data into chunks of length `bptt`. If source is equal to the example output of the batchify function, with a bptt-limit of 2, we'd get the following two Variables for i = 0:
```
┌ a g m s ┐ ┌ b h n t ┐
└ b h n t ┘ └ c i o u ┘
```
Note that despite the name of the function, the subdivison of data is not done along the batch dimension (i.e. dimension 1), since that was handled by the batchify function. The chunks are along dimension 0, corresponding to the seq_len dimension in the LSTM or RNN.

In [ ]:
!python3 -m pytest -l -v -k "ptb"

In [ ]:
!python3 -m mugrade submit "YOUR KEY HERE" -k "ptb"

## Part 7: Training a word-level language model [10 points]

Finally, you will use the `RNN` and `LSTM` components you have written to construct a language model that we will train on the Penn Treebank dataset.

First, in `python/needle/nn.py` implement `Embedding`. Consider we have a dictionary with 1000 words. Then for a word which indexes into this dictionary, we can represent this word as a one-hot vector of size 1000, and then use a linear layer to project this to a vector of some embedding size.

In `apps/models.py`, you can now implement `LanguageModel`. Your language model should consist of 

- An embedding layer (which maps word IDs to embeddings) 
- A sequence model (either RNN or LSTM)
- A linear layer (which outputs probabilities of the next word)

In `apps/simple_training.py` implement `epoch_general_ptb`, `train_ptb`, and `evaluate_ptb`.

In [ ]:
!python3 -m pytest -l -v -k "language_model_implementation"

In [ ]:
!python3 -m pytest -l -v -k "language_model_training"

In [ ]:
!python3 -m mugrade submit "YOUR KEY HERE" -k "language_model"

Now, you can train your language model on the Penn Treebank dataset:

In [ ]:
import needle as ndl
sys.path.append('./apps')
from models import LanguageModel
from simple_training import train_ptb, evaluate_ptb

device = ndl.cpu()
corpus = ndl.data.Corpus("data/ptb")
train_data = ndl.data.batchify(corpus.train, batch_size=16, device=ndl.cpu(), dtype="float32")
model = LanguageModel(30, len(corpus.dictionary), hidden_size=10, num_layers=2, seq_model='rnn', device=ndl.cpu())
train_ptb(model, train_data, seq_len=1, n_epochs=1, device=device)
evaluate_ptb(model, train_data, seq_len=40, device=device)